# TV Script Generation

In this project, I'll generate my own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  I'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network I'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [40]:
# Imports 

import helper
import problem_unittests as tests

import os
import pickle
import torch
import numpy as np
from collections import Counter
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# load in the data

data_dir = './data/Seinfeld_Scripts.txt'
input_file = os.path.join(data_dir)

with open(input_file, 'r') as f:
    text = f.read()

print('File located at directory:', input_file)

view_line_range = (0,20)
lines = text.split('\n')

print("\nSample of the data: \n")
print('\nThe lines {} to {}:'.format(*view_line_range))
print('\n'.join(lines[view_line_range[0]:view_line_range[1]]))


File located at directory: ./data/Seinfeld_Scripts.txt

Sample of the data: 


The lines 0 to 20:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! you wanna go to sleep, you wanna get up, you wanna go out a

In [3]:
# Stats about the dataset
print('Dataset Stats:\n')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

Dataset Stats:

Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143


---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, I first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [4]:
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key = word_counts.get, reverse = True) 
    int_to_vocab = {index: word for index, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: index for index, word in int_to_vocab.items()}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
Unit test for function
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    tokens = dict() # Creating a dictionary
    
    tokens['.'] = '||period||'
    tokens[','] = '||comma||'
    tokens['"'] = '||quotation_mark||'
    tokens[';'] = '||semicolon||'
    tokens['!'] = '||exclam_mark||'
    tokens['?'] = '||question_mark||'
    tokens['('] = '||left_par||'
    tokens[')'] = '||right_par||'
    tokens['-'] = '||dash||'
    tokens['\n'] = '||return||'
    
    return tokens    


"""
Unit test for function
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [6]:
# From helpers.py
SPECIAL_WORDS = {'PADDING': '<PAD>'}

def preprocess_and_save_data(dataset_path, token_lookup, create_lookup_tables):
    """
    Preprocess Text Data
    """
    input_file = os.path.join(dataset_path)
    with open(input_file, "r") as f:
        text = f.read()
    
    # Ignore notice, since we don't use it for analysing the data
    text = text[81:]

    token_dict = token_lookup()
    for key, token in token_dict.items():
        text = text.replace(key, ' {} '.format(token))
    
    text = text.lower()
    text = text.split()

    vocab_to_int, int_to_vocab = create_lookup_tables(text + list(SPECIAL_WORDS.values()))
    int_text = [vocab_to_int[word] for word in text]
    pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('preprocess.p', 'wb'))

def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('preprocess.p', mode='rb'))

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [8]:
# Checking tokenizaation
print(int_text[:100])
print()


print(int_to_vocab[1])

[24, 22, 47, 1, 1, 1, 17, 47, 22, 82, 20, 6, 1252, 545, 8782, 7189, 20, 241, 1, 149, 1, 1, 1, 84, 4, 200, 238, 149, 208, 58, 55, 135, 64, 47, 3, 24, 22, 18, 677, 208, 58, 1, 1, 1, 24, 220, 126, 2, 121, 50, 47, 86, 2, 26, 82, 22, 289, 1, 45, 82, 374, 62, 22, 289, 2, 121, 50, 47, 10, 76, 48, 149, 271, 8, 248, 191, 2, 65, 204, 27, 128, 55, 48, 1, 11, 28, 98, 2744, 172, 12, 84, 4, 1366, 3, 2, 5, 388, 248, 69, 1]

||period||


## Build the Neural Network
In this section, I'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [9]:
# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')
else:
    print('GPU is availible. YAY')

GPU is availible. YAY


## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [10]:
def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    
    n_batches = len(words) - sequence_length # leave 1 so that there is room to label last batch
    feature_tensors, target_tensors = [], []
    
    for idx in range(n_batches): # idx from 0 to len(words), increment by batch_size
        batch = words[idx:idx+sequence_length] # words from idx up to idx plus batch_size
        label = words[idx+sequence_length] # Label
        feature_tensors.append(batch)
        target_tensors.append(label)
        
    # Convert feature tensors and target tensors into an np.array
    feature_tensors = np.asarray(feature_tensors)
    target_tensors = np.asarray(target_tensors)
    
    data = TensorDataset(torch.Tensor(feature_tensors).long(), torch.Tensor(target_tensors).long())
    data_loader = torch.utils.data.DataLoader(data, shuffle = False, batch_size=batch_size)    
    
    return data_loader # return a dataloader

### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [11]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 0,  1,  2,  3,  4],
        [ 1,  2,  3,  4,  5],
        [ 2,  3,  4,  5,  6],
        [ 3,  4,  5,  6,  7],
        [ 4,  5,  6,  7,  8],
        [ 5,  6,  7,  8,  9],
        [ 6,  7,  8,  9, 10],
        [ 7,  8,  9, 10, 11],
        [ 8,  9, 10, 11, 12],
        [ 9, 10, 11, 12, 13]])
torch.Size([10])
tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [12]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
       
        # set class variables
        
        self.vocab_size = vocab_size # How many words exists in dictionary
        self.output_size = output_size # Prediction Dim
        self.embedding_dim = embedding_dim # Length of Embedded Layer
        self.hidden_dim = hidden_dim # Hidden Layer Dim
        self.n_layers = n_layers # Number of Recurrent LSTM Layers
        self.drop_prob = dropout # Dropout probability for LSTM Cell
        
        # define model layers
        
        # Vocab Size is height of matrix, emb_dim is row length
        self.embed = nn.Embedding(self.vocab_size, self.embedding_dim) 
        
        self.LSTM = nn.LSTM(self.embedding_dim, # Input
                            self.hidden_dim, # The number of features in the hidden state h
                            self.n_layers, # Num of Recurrent layers
                            dropout=self.drop_prob, # Dropout Prob
                            batch_first= True)        
        
        self.fc = nn.Linear(self.hidden_dim, self.output_size) # FC Output Layer
        
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        
        batch_size = nn_input.size(0) # 10 in our case
        
        x = self.embed(nn_input) # 1. Embedding Layer
        
        lstm_out, hidden = self.LSTM(x, hidden) # 2. LSTM layer
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim) # Stack LSTM Output for input to hidden layer
        
        fc_out = self.fc(lstm_out) # 3. Fully connected layer  
        output = fc_out.view(batch_size, -1, self.output_size) # reshape into (batch_size, seq_length, output_size)
        output = output[:, -1] # get last batch        

        # return one batch of output word scores and the hidden state
        return output, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialize hidden state with zero weights, and move to GPU if available
        
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())        
    
        return hidden


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inputs, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inputs, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [13]:
def forward_back_prop(rnn, optimizer, criterion, inputs, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # move data to GPU, if available
    if(train_on_gpu):
        inputs, target = inputs.cuda(), target.cuda()
        
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in hidden])
    
    # reset gradients
    rnn.zero_grad()
    
    output, h = rnn(inputs, h) #Foward Pass    
    
    #calculate loss and perform backpropagation
    loss = criterion(output, target)
    loss.backward() # Backprop
    
    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step() # Gradient Descent
    
    
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), h

"""
Unit test
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [14]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

## Instantiate the network

Here, we'll instantiate the network. First up, defining the hyperparameters.

* `vocab_size`: Size of our vocabulary or the range of values for our input, word tokens.
* `output_size`: Size of our desired output; the number of class scores we want to output (pos/neg).
* `embedding_dim`: Number of columns in the embedding lookup table; size of our embeddings.
* `hidden_dim`: Number of units in the hidden layers of our LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `n_layers`: Number of LSTM layers in the network. Typically between 1-3

> **Exercise:** Define the model  hyperparameters.


### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [15]:
# Data params
# Sequence Length
sequence_length = 10 # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [16]:
# Training parameters
# Number of Epochs
num_epochs = 12
# Learning Rate
learning_rate = 0.002

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim = 256
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 3000


rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
train_loader = batch_data(int_text, sequence_length, batch_size)

print(rnn)

RNN(
  (embed): Embedding(21388, 256)
  (LSTM): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=21388, bias=True)
)


### Train
In the next cell, I'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [17]:
if train_on_gpu:
    rnn.cuda()
    
# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

Training for 12 epoch(s)...
Epoch:    1/12    Loss: 4.789334419568379

Epoch:    1/12    Loss: 4.338769075155258

Epoch:    2/12    Loss: 4.064157111304147

Epoch:    2/12    Loss: 3.898628405014674

Epoch:    3/12    Loss: 3.7969913664678687

Epoch:    3/12    Loss: 3.7108417619864147

Epoch:    4/12    Loss: 3.6443472098991836

Epoch:    4/12    Loss: 3.587406188329061

Epoch:    5/12    Loss: 3.5349420061277246

Epoch:    5/12    Loss: 3.4949014871915183

Epoch:    6/12    Loss: 3.455051344568527

Epoch:    6/12    Loss: 3.4235158043702443

Epoch:    7/12    Loss: 3.3947967839018767

Epoch:    7/12    Loss: 3.366261739095052

Epoch:    8/12    Loss: 3.348009661902526

Epoch:    8/12    Loss: 3.321692591826121

Epoch:    9/12    Loss: 3.306571643458833

Epoch:    9/12    Loss: 3.285855293194453

Epoch:   10/12    Loss: 3.279528483001313

Epoch:   10/12    Loss: 3.2544228982130687

Epoch:   11/12    Loss: 3.249404517851599

Epoch:   11/12    Loss: 3.231287483294805

Epoch:   12/12    

In [18]:
# Save model
filename = './save/trained_rnn'
save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
torch.save(trained_rnn, save_filename)

C:\Users\ad55a\anaconda3\lib\site-packages\torch\serialization.py:402: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [19]:
print("Done Training")

Done Training


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** 

Attempt 1: Loss: 3.800704077005386
    
    Epochs: 15
    
    Sequence_Length: 10
    
    N_layers: 2
    
    embedding_dim = 256
    
    hidden_dim = 128
    
    lr = 0.001


Attempt 2: Loss: 3.8396477399667104
    
    Epochs: 10
    
    Sequence_Length: 7
    
    N_layers: 2
    
    embedding_dim = 256
    
    hidden_dim = 256
    
    lr = 0.003
    
Attempt 3: Loss: 3.5523485570748647 (Dropout Layer Removed)
    
    Epochs: 12
    
    Sequence_Length: 8
    
    N_layers: 2
    
    embedding_dim = 256
    
    hidden_dim = 256
    
    lr = 0.003
    
Final: Loss: 3.214442391236623 (Dropout Layer Removed)
    
    Epochs: 12
    
    Sequence_Length: 10
    
    N_layers: 2
    
    embedding_dim = 256
    
    hidden_dim = 256
    
    lr = 0.002

In [42]:
_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = os.path.splitext(os.path.basename(filename))[0] + '.pt' #Load Model

## Generate TV Script
With the network trained and saved, I'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. I'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [36]:
def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    if train_on_gpu:
        rnn.cuda()
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [38]:
# run the cell multiple times to get different results!
gen_length = 2000 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry: fairly badly.

estelle: what?

estelle: i told you, you should have been abused.

morty: you don't want to hop yourself in the bakery.

[new mac: jane?

hoyt: you got mushrooms, artie duty.

hoyt: yes, jackie, this is the nature of the stars and the reservation.

hoyt: yes.

[new witness: marcellino in latham and maligning

frank:(correcting the cereal) oh god, uh, yeah.

estelle: i think i could get my hopes to attend you to attend.

hoyt: so you can get the bags?

estelle: oh, yeah.

hoyt: you know, you were just reeling it.

hoyt: what did you get the defendants.

hoyt: i can't afford it, i don't want any money for this guy.

helen: i don't want to talk to you, and i can't afford to see perry como.

estelle: soup's, uh- what are they doing?

estelle:(whistles) i can't afford the witness.

hoyt: what do i want to do with you?

estelle: i think so.

estelle: i think it's the same... jane testimony jane.

frank: i can't. i guarantee this woman.

estelle: i was aware.

estelle: i

In [39]:
# save script to a text file
f =  open("generated_script_2.txt","w")
f.write(generated_script)
f.close()